In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

In [2]:
# Local imports
import kepler_sieve
# from asteroid_element import get_asteroids
from asteroid_data import load_ast_vectors, load_ast_data, ast_add_earth_vectors, ast_add_earth_pos, ast_add_sun_vectors
from asteroid_spline import spline_ast_vec, spline_ast_elt
from db_config import db_engine
from db_utils import sp2df

In [3]:
# df_ast_vectors = load_ast_vectors(n0=1, n1=11)

In [4]:
# df_ast_data = load_ast_data(n0=0, n1=10)

In [5]:
df_ast = load_ast_data(n0=1, n1=11)

In [6]:
ast_add_earth_pos(df_ast)

In [7]:
df_ast

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M,earth_qx,earth_qy,earth_qz
0,69120000,1,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583,2.768788,0.077540,0.185062,1.408422,1.237946,-0.601468,-0.517779,-0.880033,-0.483797,-0.000050
1,69125760,1,48004.0,-1.192813,2.275319,0.289164,-0.009377,-0.005620,0.001563,2.768752,0.077525,0.185061,1.408417,1.237928,-0.584374,-0.502840,-0.845941,-0.543195,-0.000054
2,69131520,1,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543,2.768714,0.077510,0.185060,1.408411,1.237912,-0.567257,-0.487900,-0.807882,-0.600059,-0.000056
3,69137280,1,48012.0,-1.267161,2.229116,0.301507,-0.009208,-0.005929,0.001522,2.768677,0.077495,0.185059,1.408406,1.237895,-0.550118,-0.472959,-0.766061,-0.654113,-0.000055
4,69143040,1,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501,2.768639,0.077479,0.185057,1.408400,1.237879,-0.532956,-0.458018,-0.720707,-0.705117,-0.000054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37505,90696960,10,62984.0,0.728575,3.372451,0.098143,-0.008660,0.001518,-0.000541,3.147844,0.104631,0.066710,-1.344509,-0.805467,3.506618,-2.696021,-0.804421,-0.605790,0.000125
37506,90702720,10,62988.0,0.693892,3.378329,0.095975,-0.008681,0.001421,-0.000543,3.147839,0.104631,0.066711,-1.344511,-0.805428,3.516782,-2.683744,-0.762203,-0.659471,0.000130
37507,90708480,10,62992.0,0.659129,3.383816,0.093796,-0.008700,0.001323,-0.000546,3.147833,0.104631,0.066711,-1.344513,-0.805389,3.526955,-2.671469,-0.716489,-0.710085,0.000132
37508,90714240,10,62996.0,0.624289,3.388913,0.091606,-0.008719,0.001225,-0.000549,3.147828,0.104632,0.066711,-1.344515,-0.805349,3.537136,-2.659194,-0.667499,-0.757430,0.000132


In [8]:
# Sample dates to interpolate
mjd = np.arange(59000, 59100, 0.25)

In [9]:
# Spline vectors directly
df_vec = spline_ast_vec(df_ast=df_ast, mjd=mjd)

In [10]:
df_vec

,AsteroidID,mjd,qx,qy,qz,vx,vy,vz
0,1,59000.00,2.200915,-1.931843,-0.467561,0.006340,0.007130,-0.000945
1,1,59000.25,2.202500,-1.930060,-0.467797,0.006334,0.007136,-0.000943
2,1,59000.50,2.204082,-1.928275,-0.468033,0.006328,0.007141,-0.000942
3,1,59000.75,2.205664,-1.926489,-0.468268,0.006322,0.007147,-0.000941
4,1,59001.00,2.207243,-1.924702,-0.468503,0.006316,0.007152,-0.000939
...,...,...,...,...,...,...,...,...
3995,10,59098.75,-0.902277,3.228486,-0.009105,-0.008493,-0.003236,-0.000602
3996,10,59099.00,-0.904400,3.227676,-0.009255,-0.008491,-0.003243,-0.000602
3997,10,59099.25,-0.906522,3.226865,-0.009406,-0.008489,-0.003249,-0.000602
3998,10,59099.50,-0.908644,3.226052,-0.009556,-0.008487,-0.003255,-0.000602


In [13]:
# Spline orbital elements
df_elt = spline_ast_elt(df_ast=df_ast, mjd=mjd)

# Add sun position
ast_add_earth_pos(df_ast=df_elt)
ast_add_sun_vectors(df_ast=df_elt)

In [14]:
df_elt

,AsteroidID,mjd,a,e,inc,Omega,omega,f,M,earth_qx,earth_qy,earth_qz,sun_qx,sun_qy,sun_qz,sun_vx,sun_vy,sun_vz
0,1,59000.00,2.767657,0.077557,0.184806,1.401272,1.286860,-3.401546,2.839411,-0.356169,-0.944135,0.000100,-0.356169,-0.944135,0.000100,0.015852,-0.006019,4.071973e-07
1,1,59000.25,2.767654,0.077558,0.184806,1.401272,1.286859,-3.400738,2.840346,-0.352203,-0.945632,0.000100,-0.352203,-0.945632,0.000100,0.015876,-0.005952,4.450600e-07
2,1,59000.50,2.767652,0.077559,0.184806,1.401271,1.286858,-3.399930,2.841282,-0.348231,-0.947111,0.000100,-0.348231,-0.947111,0.000100,0.015900,-0.005884,4.832676e-07
3,1,59000.75,2.767650,0.077560,0.184806,1.401271,1.286856,-3.399122,2.842217,-0.344252,-0.948574,0.000100,-0.344252,-0.948574,0.000100,0.015924,-0.005816,5.216532e-07
4,1,59001.00,2.767647,0.077561,0.184806,1.401270,1.286855,-3.398314,2.843153,-0.340268,-0.950019,0.000100,-0.340268,-0.950019,0.000100,0.015948,-0.005748,5.600449e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,10,59098.75,3.142588,0.112017,0.066875,-1.340438,-0.830428,-2.261876,-2.089078,0.964427,-0.265909,0.000092,0.964427,-0.265909,0.000092,0.004368,0.016489,-9.391485e-07
3996,10,59099.00,3.142589,0.112016,0.066875,-1.340438,-0.830428,-2.262896,-2.088307,0.965510,-0.261785,0.000092,0.965510,-0.261785,0.000092,0.004298,0.016508,-9.648162e-07
3997,10,59099.25,3.142589,0.112016,0.066875,-1.340438,-0.830429,-2.263968,-2.087535,0.966576,-0.257655,0.000092,0.966576,-0.257655,0.000092,0.004228,0.016528,-9.893583e-07
3998,10,59099.50,3.142590,0.112016,0.066875,-1.340438,-0.830429,-2.265008,-2.086763,0.967625,-0.253521,0.000092,0.967625,-0.253521,0.000092,0.004158,0.016547,-1.012715e-06
